<a href="https://colab.research.google.com/github/Hemasaiprasanna/FMML_ASSIGNMENT_AND_LABS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Reduced Variance: Averaging results from multiple splits helps reduce the impact of randomness and variability in your data. In a single split, the specific data points chosen for the training and validation sets can greatly affect the reported accuracy. By using multiple splits and averaging, you get a more robust estimate of your model's performance.

Better Generalization: Cross-validation provides a better estimate of how well your model will generalize to new, unseen data. It helps detect issues like overfitting or underfitting that might be specific to a particular data split.

More Comprehensive Evaluation: By repeating the training and validation process with different subsets of the data, you gain insights into how your model performs across different subsets of the data, which can be valuable in understanding its strengths and weaknesses.

Reliable Metrics: When you average the results of k-fold cross-validation, you can compute more reliable performance metrics, such as mean accuracy and standard deviation. These metrics provide a clearer picture of the model's expected performance and its variability.

Variability in Data: Cross-validation helps reduce the impact of variability in your dataset by averaging over multiple data splits. However, it can only provide an estimate based on the data you have. The true test data, which represents unseen data, might have characteristics that differ from your training and validation sets.

Data Leakage: While cross-validation helps prevent data leakage to some extent by using separate training and validation sets in each fold, it doesn't completely eliminate the risk of data leakage. Some leakage can still occur if there are dependencies between the data points that persist across different folds.

Model Variability: Cross-validation provides estimates of your model's performance based on the specific training-validation splits. If you have a small dataset or a highly variable model, these estimates might still have some variance.

Hyperparameters and Model Selection: Cross-validation helps you choose hyperparameters and assess model performance, but the final test accuracy might depend on the specific hyperparameters you select after cross-validation. If the chosen hyperparameters do not generalize well to the true test data, your test accuracy could differ.

Training Iterations (Epochs) in Model Training:

In deep learning and other iterative training processes, increasing the number of training iterations (epochs) can improve the model's performance up to a point. This allows the model to learn better representations from the data and potentially converge to a more optimal solution.
However, there is a trade-off. If you increase the number of iterations excessively, the model might overfit the training data, resulting in poor generalization to unseen data (i.e., the test data).
The impact of the number of training iterations on your estimate of model performance is indirect. It affects how well your model can fit the training data, which, in turn, affects the results of cross-validation.
Iterations in Optimization Algorithms:

In optimization algorithms like gradient descent, the number of iterations directly impacts the optimization process. More iterations can lead to a closer approximation of the optimal solution if the optimization problem is not converging quickly.
Increasing iterations in optimization may help find a better model parameter configuration, which can positively impact the results of cross-validation.
Iterations in Cross-Validation (Folds):

In k-fold cross-validation, the number of iterations (k) represents how many times the data is split into training and validation sets. Typically, a higher number of iterations (higher k) results in a more reliable estimate of model performance because it reduces the variability associated with random data splits.
However, there are diminishing returns with increasing k. Very high values of k can be computationally expensive and may not significantly improve the estimate beyond a certain point.

Increasing the number of iterations can help mitigate some of the challenges associated with very small training or validation datasets, but it is not a complete solution, and there are limitations to what increased iterations can achieve in such scenarios. Here's a breakdown of how iterations can impact the situation:

Small Training Dataset:

When you have a small training dataset, increasing the number of training iterations (epochs) can allow your model to see and learn from the available data more thoroughly. This can potentially help the model fit the training data better.

However, there are limits to how much increasing iterations can compensate for a small training dataset. If the dataset is too small, the risk of overfitting increases as the model may memorize the training examples instead of learning generalizable patterns. Overfitting can lead to poor performance on new, unseen data (including the validation set).

Small Validation Dataset:

A small validation dataset can make it challenging to reliably estimate your model's performance. With limited validation examples, the estimate of model performance becomes more prone to random variability.

Increasing the number of training iterations doesn't directly address the issue of a small validation dataset. It primarily impacts how well your model generalizes from the training data, but the validation dataset size remains fixed.

To address these challenges, consider the following strategies:

Data Augmentation: If you have a small training dataset, you can use data augmentation techniques to artificially increase the effective size of your training data. Data augmentation involves applying various transformations to the existing data (e.g., rotation, scaling, cropping) to create additional training examples.

Regularization: Employ regularization techniques such as dropout, L1 or L2 regularization, or early stopping to prevent overfitting, especially when dealing with a small training dataset.

Cross-Validation: When dealing with a small validation dataset, you can still perform cross-validation, but it might be challenging to split the data into many folds. Consider using leave-one-out cross-validation (LOOCV) or a variation like stratified k-fold with a smaller value of k.

Model Selection: If you have a small validation dataset, use it primarily for hyperparameter tuning and model selection rather than obtaining a highly precise estimate of performance. After selecting the best model and hyperparameters, evaluate the model on a separate, larger test dataset to obtain a more reliable estimate of its generalization performance.